In [29]:
#We need these in this file:
import pandas as pd
from sklearn import svm
from sklearn.metrics import classification_report,accuracy_score
import numpy as np # linear algebra
import get_lodging_scores
import load_read_name_extractor as lrne
import SVM_classifier_general as svm_general
import pickle
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV
import glob
import tracemalloc


In [30]:
data_paths = [
"Features\hog_features_(64, 128, 3)_cells_(8, 8)_block_(1, 1)_norm_L2.npy",
"Features\hog_features_(64, 128, 3)_cells_(8, 8)_block_(2, 2)_norm_L2.npy",
"Features\hog_features_(64, 128, 3)_cells_(8, 8)_block_(3, 3)_norm_L2.npy",
"Features\hog_features_(64, 128, 3)_cells_(8, 8)_block_(4, 4)_norm_L2.npy",
"Features\hog_features_(96, 192, 3)_cells_(8, 8)_block_(1, 1)_norm_L2.npy",
"Features\hog_features_(96, 192, 3)_cells_(8, 8)_block_(2, 2)_norm_L2.npy",
"Features\hog_features_(96, 192, 3)_cells_(8, 8)_block_(3, 3)_norm_L2.npy",
"Features\img_names_(64, 128, 3)_cells_(8, 8)_block_(1, 1)_norm_L2.npy",
"Features\img_names_(64, 128, 3)_cells_(8, 8)_block_(2, 2)_norm_L2.npy",
"Features\img_names_(64, 128, 3)_cells_(8, 8)_block_(3, 3)_norm_L2.npy",
"Features\img_names_(64, 128, 3)_cells_(8, 8)_block_(4, 4)_norm_L2.npy",
"Features\img_names_(96, 192, 3)_cells_(8, 8)_block_(1, 1)_norm_L2.npy",
"Features\img_names_(96, 192, 3)_cells_(8, 8)_block_(2, 2)_norm_L2.npy",
"Features\img_names_(96, 192, 3)_cells_(8, 8)_block_(3, 3)_norm_L2.npy"]

In [31]:
features_paths = data_paths[0:7]
names_paths = data_paths[7:14]

In [32]:
features_paths =    [
                    "Features\hog_features_(64, 128, 3)_cells_(8, 8)_block_(5, 5)_norm_L2.npy",
                    "Features\hog_features_(64, 128, 3)_cells_(8, 8)_block_(6, 6)_norm_L2.npy",
                    "Features\hog_features_(96, 192, 3)_cells_(8, 8)_block_(5, 5)_norm_L2.npy",
                    "Features\hog_features_(96, 192, 3)_cells_(8, 8)_block_(6, 6)_norm_L2.npy"
                    ]
names_paths =       [
                    "Features\img_names_(64, 128, 3)_cells_(8, 8)_block_(5, 5)_norm_L2.npy",
                    "Features\img_names_(64, 128, 3)_cells_(8, 8)_block_(6, 6)_norm_L2.npy",
                    "Features\img_names_(96, 192, 3)_cells_(8, 8)_block_(5, 5)_norm_L2.npy",
                    "Features\img_names_(96, 192, 3)_cells_(8, 8)_block_(6, 6)_norm_L2.npy"
                    ]

In [33]:
data_paths_auto = glob.glob("Features/*npy")
size = int(len(data_paths_auto)/2)
features_paths = data_paths_auto[0:size]
names_paths = data_paths_auto[size:len(data_paths_auto)]

In [34]:
features_paths = [
 'Features\\hog_features_(155, 543, 3)_cells_(8, 8)_block_(1, 1)_norm_L2.npy',
 'Features\\hog_features_(155, 543, 3)_cells_(8, 8)_block_(2, 2)_norm_L2.npy',
 'Features\\hog_features_(155, 543, 3)_cells_(8, 8)_block_(3, 3)_norm_L2.npy',
 'Features\\hog_features_(155, 543, 3)_cells_(8, 8)_block_(4, 4)_norm_L2.npy',
 'Features\\hog_features_(155, 543, 3)_cells_(8, 8)_block_(5, 5)_norm_L2.npy',
 'Features\\hog_features_(155, 543, 3)_cells_(8, 8)_block_(6, 6)_norm_L2.npy']

In [35]:
names_paths = [
 'Features\\img_names_(155, 543, 3)_cells_(8, 8)_block_(1, 1)_norm_L2.npy',
 'Features\\img_names_(155, 543, 3)_cells_(8, 8)_block_(2, 2)_norm_L2.npy',
 'Features\\img_names_(155, 543, 3)_cells_(8, 8)_block_(3, 3)_norm_L2.npy',
 'Features\\img_names_(155, 543, 3)_cells_(8, 8)_block_(4, 4)_norm_L2.npy',
 'Features\\img_names_(155, 543, 3)_cells_(8, 8)_block_(5, 5)_norm_L2.npy',
 'Features\\img_names_(155, 543, 3)_cells_(8, 8)_block_(6, 6)_norm_L2.npy']

In [36]:
features_load = np.load(features_paths[5], allow_pickle=True)
features_load[0].shape

(42768,)

In [37]:
def save_results(results, path):
    path = path.replace("Features\\", "")
    path = "results/" + path.replace(".npy", ".txt")
    with open(path, 'a') as f:
        f.write(results)
        f.write("\n")

In [38]:
def load_labels(bin_size):
    labels = get_lodging_scores.get_labels(bin_size)
    #COnverts into set (no dublicates in set)
    my_set = set(map(tuple, labels))
    # convert set back to list of arrays
    labels = list(map(np.array, my_set))
    return labels

In [39]:
def partition_data(data_frame, percentage):
    partition = int(len(data_frame)*percentage/100)
    print(len(data_frame.shape), 2)
    print(len(data_frame[0].shape), 2)
    x_train, x_test = data_frame[:partition,:-1],  data_frame[partition:,:-1]
    y_train, y_test = data_frame[:partition,-1:].ravel() , data_frame[partition:,-1:].ravel()
    return x_train, x_test, y_train, y_test

In [40]:
def save_model(filename, clf):
    filename = filename.replace("Features\\", "")
    filename = "model/" + filename.replace(".npy", ".sav")
    pickle.dump(clf, open(filename, 'wb'))

In [41]:
def run_mult_models(features_paths, names_paths, bin_size = 10, svm_kernel="rbf", class_weights = None):
    for i in range (len(features_paths)):
        

        #laod features and names
        features_load = np.load(features_paths[i], allow_pickle=True)
        names_load = np.load(names_paths[i], allow_pickle=True)
        #load labels
        labels = load_labels(bin_size)
        #convert names into [flight_folder, ROI]
        names = lrne.it_name_extract_labels_from_img_jpeg(names_load)
        names_load = None
        #match labels with feature names
        index = svm_general.match_pic_label_to_names(features_load, labels, names)
        data_frame = [list(row) for row in features_load]
        features_load = None

        for i in range(len(index)):
            data_frame[index[i]].append(labels[i][3])
        labels = None
        names = None
        print(len(data_frame), 0)
        print(len(data_frame[0]), 0)

        #shuffle data and convert to array
        #np.random.shuffle(data_frame)

        #partition data
        percentage = 80
        data_frame = np.array(data_frame)
        x_train, x_test, y_train, y_test = partition_data(data_frame, percentage)
        data_frame = None
        features_aar = None

        #create svm
        clf = svm.SVC(kernel=svm_kernel, class_weight=class_weights)
        #fit model with training data
        clf.fit(x_train,y_train)

        #predict data
        y_pred = clf.predict(x_test)

        #Save the found model
        n_bins = int(np.floor(100/bin_size))
        path = "Kernel_" + svm_kernel + "_Nbins_" + str(n_bins) + "_ClassWeights_" + str(class_weights) + "_" + features_paths[i]
        save_model(features_paths[i], clf)

        #Save the results
        results = "Accuracy: "+str(accuracy_score(y_test, y_pred)) + "\n" + classification_report(y_test, y_pred)
        save_results(results, path)
        tracemalloc.stop()

In [42]:
run_mult_models([features_paths[0]], names_paths, bin_size = 10, svm_kernel="rbf")

4915 0
42769 0
4915 1
42769 1
4915 2
42769 2
1 2


C:\Users\marti\AppData\Local\Temp\ipykernel_3204\310217235.py:29: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  data_frame = np.array(data_frame)


AttributeError: 'list' object has no attribute 'shape'